In [2]:

from linkers.sparse import SparseLinker
from candidate import make_phrases

dataset_fpaths = ["datasets/dbpedia.tsv", 
                  "datasets/kore50.tsv", "datasets/n3-reuters-128.tsv"]

sl = SparseLinker("data/test99")
# sl.train(dataset_fpaths)
context = "Madonna is a great music  signer and lives near West Holywood in LA. adonna Louise Ciccone (/tʃɪˈkoʊni/; born August 16, 1958) is an American singer, songwriter, actress, and businesswoman. Referred to as the Queen of Pop since the 1980s, Madonna is known for pushing the boundaries of lyrical content in mainstream popular music, as well as visual imagery in music videos and on stage. She has also frequently reinvented both her music and image while maintaining autonomy within the recording industry. Besides sparking controversy, her works have bee "
phrases = ["Madonna"]
linked_phrases = sl.link(context, make_phrases(phrases))
print(linked_phrases)

Parameters:
-  use_overlap: True
- tfidf: True
- description: 
- num_phrases: 997
- shape: [103227, 218026]
Loaded in 13.55 sec.
Retrieved 234 candidates for 'Madonna'
>>> Phrase(text='Madonna', beg=1, end=7, subj='http://Madonna') Candidate(score=0.84286408757297537, name='Madonna', link='http://dbpedia.org/resource/Madonna_(entertainer)', wiki='en.wikipedia.org/wiki/Madonna_(entertainer)', types=['Person'], names=['Myself', 'The Material Girl'], uris={'en.wikipedia.org/wiki/Madonna_(entertainer)', 'evil_https://www.discogs.com/artist/8760-Madonna'}, text='Madonna . Myself . The Material Girl . Madonna Louise Ciccone (born August 16, 1958) is an American singer, songwriter, actress, and businesswoman. Referred to as the "Queen of Pop" since the 1980s, Madonna is known for pushing the boundaries of lyrical content in mainstream popular music, as well as visual imagery in music videos and on stage. She has also frequently reinvented both her music and image while maintaining autonomy wi

In [ ]:
ttl_fpath = "data/sparse-15254408083571131-request.ttl"

import codecs

with codecs.open(ttl_fpath, "r", "utf-8") as ttl_f:
    ttl = ttl_f.read()
    
print(sl.link_ttl(ttl))

In [ ]:
from candidate import Phrase
p = Phrase("Hypertext Markup Language","sdf","sdfff","sdff")
sl._default_phrase(p) in sl._phrase2candidates

In [12]:
import numpy as np
sims = np.ndarray([1,20])

4.641121673612072e-310

0 1
1 2
2 3


In [ ]:
from linkers.baseline import BaselineLinker
from candidate import Phrase

context = "San Francisco said the visit would serve as a cornerstone for future interaction between players and coaches from the Nets and young Russians, with the aim of developing basketball in Russia, where the sport is a distant third in popularity behind soccer and hockey."
phrases = "San Francisco"

phrases =  [Phrase(phrase.strip(), 1, len(phrase.strip()), "http://" + phrase.strip())
                   for phrase in phrases.split(",")]
bl = BaselineLinker()

for phrase, candidate in bl.link(context, phrases):
    print(phrase.text, candidate)

In [ ]:
from linkers.baseline import BaselineLinker
from candidate import Phrase
from pandas import read_csv 

dataset_fpath = "datasets/dbpedia.tsv"

df = read_csv(dataset_fpath, sep="\t", encoding="utf-8")
bl = BaselineLinker()

for i, row in df.iterrows():
    phrases =  [Phrase(phrase.strip(), 1, len(phrase.strip()), "http://" + phrase.strip())
                       for phrase in row.targets.split(",")]
        
    print("\n\n{}\n".format(row.context))
    
    for phrase, candidate in bl.link(row.context, phrases):
        link = candidate.link if candidate else ""
        print(phrase.text, link)

In [ ]:
%load_ext autoreload
%autoreload 2